In [ ]:
class State():
    raw_question: str
    next_step: 

    answer: str

In [ ]:
from typing_extensions import Literal
from langgraph.graph import MessagesState
from pydantic import BaseModel
from langchain_openai import ChatOpenAI

# =========================
# 1. State 정의
# =========================
class State(MessagesState, total=False):
    question: str                           # 전처리 질문 # 확인필요
    category: str                           # 전처리된 카테고리 #확인필요
    is_hr_question: Literal["yes", "no"]    # HR 여부
    next_step: Literal["router2", "reject"] # 다음 노드 방향

# =========================
# 2. Pydantic 모델 정의
# =========================
class HRClassification(BaseModel):
    is_hr_question: Literal["yes", "no"]
    next_step: Literal["router2", "reject"]

# =========================
# 3. HR NODE (with_structured_output)
# =========================
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

def hr_node(state: State) -> State:
    """HR 관련 질문인지 판별하는 노드 (Structured Output)"""
    structured_llm = llm.with_structured_output(HRClassification)

    prompt = f"""
    당신은 "가이다 플레이 스튜디오(GPS)"의 HR 안내 챗봇입니다.  
    회사 내부 직원의 질문이 HR과 관련된 질문인지 아닌지를 판별하세요.  

    ### 참고 정보
    - 카테고리: "{state['category']}"
    - 질문: "{state['question']}"

    ### 출력 형식 (반드시 JSON):
    {{
      "is_hr_question": "yes" | "no",
      "next_step": "router2" | "reject"
    }}
    """

    try:
        result: HRClassification = structured_llm.invoke(prompt)
    except Exception:
        # fallback: HR 아님 처리
        return {**state, "is_hr_question": "no", "next_step": "reject"}

    return {
        **state,
        "is_hr_question": result.is_hr_question,
        "next_step": result.next_step,
    }

# =========================
# 4. HR Router
# =========================
def route_after_hr(state: State) -> str:
    """HR 판별 결과에 따라 다음 노드 결정"""
    return state["next_step"]

# =========================
# 5. Reject Node
# =========================
def route_after_hr(state: State) -> str:
    """HR 판별 결과에 따라 다음 노드 결정"""
    return "router2" if state["next_step"] == "router2" else "reject"
